In [1]:
import re
import traceback
import librosa
import sklearn
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join

In [2]:
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)

In [3]:

def extract_feature(path):
    id = 0 
    path+='/'
    feature_set = pd.DataFrame() 
    file_data = [f.strip() for f in listdir(path) if isfile(join(path, f))]
    file_data = [x for x in file_data if re.match(r'.*\.mp3',x, re.I) is not None]
    n_songs = len(file_data)
    
    songname_vector = pd.Series()
    tempo_vector = pd.Series()
    duration_vector = pd.Series()
    rms_mean = pd.Series()
    rms_var = pd.Series()
    zcr_mean = pd.Series()
    zcr_var = pd.Series()
    cent_mean = pd.Series()
    cent_var = pd.Series()
    spec_bw_mean = pd.Series()
    spec_bw_var = pd.Series()
    rolloff_mean = pd.Series()
    rolloff_var = pd.Series()
    frame_mean = pd.Series()
    frame_var = pd.Series()
    
    chroma_stft_mean = np.zeros([n_songs,12])
    chroma_stft_var = np.zeros([n_songs,12])
    chroma_cq_mean = np.zeros([n_songs,12])
    chroma_cq_var = np.zeros([n_songs,12])
    chroma_cens_mean = np.zeros([n_songs,12])
    chroma_cens_var = np.zeros([n_songs,12])
    mel_mean = np.zeros([n_songs,128])
    mel_var = np.zeros([n_songs,128])
    mfcc_mean = np.zeros([n_songs,20])
    mfcc_var = np.zeros([n_songs,20])
    mfcc_delta_mean = np.zeros([n_songs,20])
    mfcc_delta_var = np.zeros([n_songs,20])
    
    contrast_mean = np.zeros([n_songs,7])
    contrast_var = np.zeros([n_songs,7])
    poly_mean = np.zeros([n_songs,3])
    poly_var = np.zeros([n_songs,3])
    tonnetz_mean = np.zeros([n_songs,6])
    tonnetz_var = np.zeros([n_songs,6])
    harm_mean = np.zeros([n_songs,12])
    harm_var = np.zeros([n_songs,12])
    perc_mean = np.zeros([n_songs,12])
    perc_var = np.zeros([n_songs,12])
    # --------------------------------------------------------------------------------------
    for file_name in file_data:
        try:
            songname = path + file_name
            y, sr = librosa.load(songname, duration=60)
            S = np.abs(librosa.stft(y))
            # ----------------------------------------------------------------------------------
            tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
            rms = librosa.feature.rms(y=y)
            zcr = librosa.feature.zero_crossing_rate(y)
            cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
            frames_to_time = librosa.frames_to_time(onset_frames[:20], sr=sr)

            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            chroma_cq = librosa.feature.chroma_cqt(y=y, sr=sr)
            chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr)
            melspectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            mfcc_delta = librosa.feature.delta(mfcc)

            contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
            poly_features = librosa.feature.poly_features(S=S, sr=sr, order=2)
            harmonic = librosa.effects.harmonic(y)
            percussive = librosa.effects.percussive(y)
            harm_chroma_stft = librosa.feature.chroma_stft(y=harmonic, sr=sr)
            perc_chroma_stft = librosa.feature.chroma_stft(y=percussive, sr=sr)
            tonnetz = librosa.feature.tonnetz(y=harmonic, sr=sr)
            # ----------------------------------------------------------------------------------
            songname_vector.at[id] = file_name 
            tempo_vector.at[id] = tempo 
            duration_vector.at[id] = librosa.get_duration(filename=songname) 
            rms_mean.at[id] = np.mean(rms) 
            rms_var.at[id] = np.var(rms)
            zcr_mean.at[id] = np.mean(zcr) 
            zcr_var.at[id] = np.var(zcr)
            cent_mean.at[id] = np.mean(cent) 
            cent_var.at[id] = np.var(cent)
            spec_bw_mean.at[id] = np.mean(spec_bw) 
            spec_bw_var.at[id] = np.var(spec_bw)
            rolloff_mean.at[id] = np.mean(rolloff) 
            rolloff_var.at[id] = np.var(rolloff)
            frame_mean.at[id] = np.mean(frames_to_time) 
            frame_var.at[id] = np.var(frames_to_time)

            chroma_stft_mean[id] = np.mean(chroma_stft, axis=1) 
            chroma_stft_var[id] = np.var(chroma_stft, axis=1)
            chroma_cq_mean[id] = np.mean(chroma_cq, axis=1) 
            chroma_cq_var[id] = np.var(chroma_cq, axis=1)
            chroma_cens_mean[id] = np.mean(chroma_cens, axis=1) 
            chroma_cens_var[id] = np.var(chroma_cens, axis=1)
            mel_mean[id] = np.mean(melspectrogram, axis=1) 
            mel_var[id] = np.var(melspectrogram, axis=1)
            mfcc_mean[id] = np.mean(mfcc, axis=1) 
            mfcc_var[id] = np.var(mfcc, axis=1)
            mfcc_delta_mean[id] = np.mean(mfcc_delta, axis=1) 
            mfcc_delta_var[id] = np.var(mfcc_delta, axis=1)

            contrast_mean[id] = np.mean(contrast, axis=1) 
            contrast_var[id] = np.var(contrast, axis=1)
            poly_mean[id] = np.mean(poly_features, axis=1) 
            poly_var[id] = np.var(poly_features, axis=1)
            tonnetz_mean[id] = np.mean(tonnetz, axis=1) 
            tonnetz_var[id] = np.var(tonnetz, axis=1)
            harm_mean[id] = np.mean(harm_chroma_stft, axis=1) 
            harm_var[id] = np.var(harm_chroma_stft, axis=1)
            perc_mean[id] = np.mean(perc_chroma_stft, axis=1) 
            perc_var[id] = np.var(perc_chroma_stft, axis=1)
            # ----------------------------------------------------------------------------------
            id = id+1
            print('COMPLETED: ',id,'/',n_songs,' : ',file_name)
        except:
            id = id+1
            print('#########   ERROR: ',id,'/',n_songs,' : ',file_name)
            traceback.print_exc()
        
    # --------------------------------------------------------------------------------------
    feature_set['song_name'] = songname_vector
    feature_set['tempo'] = tempo_vector
    feature_set['duration'] = duration_vector
    feature_set['rms_mean'] = rms_mean
    feature_set['rms_var'] = rms_var
    feature_set['zcr_mean'] = zcr_mean
    feature_set['zcr_var'] = zcr_var
    feature_set['cent_mean'] = cent_mean
    feature_set['cent_var'] = cent_var
    feature_set['spec_bw_mean'] = spec_bw_mean
    feature_set['spec_bw_var'] = spec_bw_var
    feature_set['rolloff_mean'] = rolloff_mean
    feature_set['rolloff_var'] = rolloff_var
    feature_set['frame_mean'] = frame_mean
    feature_set['frame_var'] = frame_var

    chroma_stft_mean = pd.DataFrame(data=chroma_stft_mean, columns = ['chroma_stft_mean'+format(a,'02') for a in range(1, 1+chroma_stft_mean.shape[1])])
    chroma_stft_var = pd.DataFrame(data=chroma_stft_var, columns = ['chroma_stft_var'+format(a,'02') for a in range(1, 1+chroma_stft_var.shape[1])])
    chroma_cq_mean = pd.DataFrame(data=chroma_cq_mean, columns = ['chroma_cq_mean'+format(a,'02') for a in range(1, 1+chroma_cq_mean.shape[1])])
    chroma_cq_var = pd.DataFrame(data=chroma_cq_var, columns = ['chroma_cq_var'+format(a,'02') for a in range(1, 1+chroma_cq_var.shape[1])])
    chroma_cens_mean = pd.DataFrame(data=chroma_cens_mean, columns = ['chroma_cens_mean'+format(a,'02') for a in range(1, 1+chroma_cens_mean.shape[1])])
    chroma_cens_var = pd.DataFrame(data=chroma_cens_var, columns = ['chroma_cens_var'+format(a,'02') for a in range(1, 1+chroma_cens_var.shape[1])])
    mel_mean = pd.DataFrame(data=mel_mean, columns = ['mel_mean'+format(a,'03') for a in range(1, 1+mel_mean.shape[1])])
    mel_var = pd.DataFrame(data=mel_var, columns = ['mel_var'+format(a,'03') for a in range(1, 1+mel_var.shape[1])])
    mfcc_mean = pd.DataFrame(data=mfcc_mean, columns = ['mfcc_mean'+format(a,'02') for a in range(1, 1+mfcc_mean.shape[1])])
    mfcc_var = pd.DataFrame(data=mfcc_var, columns = ['mfcc_var'+format(a,'02') for a in range(1, 1+mfcc_var.shape[1])])
    mfcc_delta_mean = pd.DataFrame(data=mfcc_delta_mean, columns = ['mfcc_delta_mean'+format(a,'02') for a in range(1, 1+mfcc_delta_mean.shape[1])])
    mfcc_delta_var = pd.DataFrame(data=mfcc_delta_var, columns = ['mfcc_delta_var'+format(a,'02') for a in range(1, 1+mfcc_delta_var.shape[1])])

    contrast_mean = pd.DataFrame(data=contrast_mean, columns = ['contrast_mean'+format(a,'02') for a in range(1, 1+contrast_mean.shape[1])])
    contrast_var = pd.DataFrame(data=contrast_var, columns = ['contrast_var'+format(a,'02') for a in range(1, 1+contrast_var.shape[1])])
    poly_mean = pd.DataFrame(data=poly_mean, columns = ['poly_mean'+format(a,'02') for a in range(1, 1+poly_mean.shape[1])])
    poly_var = pd.DataFrame(data=poly_var, columns = ['poly_var'+format(a,'02') for a in range(1, 1+poly_var.shape[1])])
    tonnetz_mean = pd.DataFrame(data=tonnetz_mean, columns = ['tonnetz_mean'+format(a,'02') for a in range(1, 1+tonnetz_mean.shape[1])])
    tonnetz_var = pd.DataFrame(data=tonnetz_var, columns = ['tonnetz_var'+format(a,'02') for a in range(1, 1+tonnetz_var.shape[1])])
    harm_mean = pd.DataFrame(data=harm_mean, columns = ['harm_mean'+format(a,'02') for a in range(1, 1+harm_mean.shape[1])])
    harm_var = pd.DataFrame(data=harm_var, columns = ['harm_var'+format(a,'02') for a in range(1, 1+harm_var.shape[1])])
    perc_mean = pd.DataFrame(data=perc_mean, columns = ['perc_mean'+format(a,'02') for a in range(1, 1+perc_mean.shape[1])])
    perc_var = pd.DataFrame(data=perc_var, columns = ['perc_var'+format(a,'02') for a in range(1, 1+perc_var.shape[1])])

    feature_set = pd.concat([feature_set,chroma_stft_mean,chroma_stft_var,chroma_cq_mean,chroma_cq_var,chroma_cens_mean,chroma_cens_var,mel_mean,mel_var,
               mfcc_mean,mfcc_var,mfcc_delta_mean,mfcc_delta_var,contrast_mean,contrast_var,poly_mean,poly_var,tonnetz_mean,tonnetz_var,
               harm_mean,harm_var,perc_mean,perc_var], axis=1)
    # --------------------------------------------------------------------------------------
    feature_set.to_csv('Song_features.csv')
    feature_set.to_json('Song_features.json')
    

In [4]:
# Extracting Feature Function Call
extract_feature('music')

C:\Users\pc\Anaconda3\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


COMPLETED:  1 / 1008  :  'Cos I'm your lady.mp3


C:\Users\pc\Anaconda3\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


COMPLETED:  2 / 1008  :  -Songs-of-War-The-North-Remembers-Game-of-Thrones-tribute-by-Chris-Yenney-.mp3
COMPLETED:  3 / 1008  :  004 Enna Sona.mp3
COMPLETED:  4 / 1008  :  005 SWIKAR KIYA MAINE = CHAND KE PAAS JO.mp3
COMPLETED:  5 / 1008  :  007. Kabhi To Nazar Milao = Kabhi To Nazar Milao.mp3
COMPLETED:  6 / 1008  :  009 DIL- A - NADAN = CHANDANI RAAT MAIN.mp3
COMPLETED:  7 / 1008  :  01 - Besabriyaan - DownloadMing.SE.mp3
COMPLETED:  8 / 1008  :  01 - Bezubaan - [rKmania.me].mp3
COMPLETED:  9 / 1008  :  01 - Boom Boom (Lip Lock) (www.SongsLover.pk).mp3
COMPLETED:  10 / 1008  :  01 - Deewana Kar Raha Hai - [rKmania.me].mp3
COMPLETED:  11 / 1008  :  01 - Ender's Game - 01 Ender's War (OST 2013 HD).mp3
COMPLETED:  12 / 1008  :  01 - Party On My Mind [MusikMaza.Com].mp3
COMPLETED:  13 / 1008  :  01 aaj jane ki jidna.MP3
COMPLETED:  14 / 1008  :  01-SAJDA.MP3
COMPLETED:  15 / 1008  :  02 - Be Intehaan [MusikMaza.Com].mp3
COMPLETED:  16 / 1008  :  02 - Ender's Game - 02 Stay Down (OST 2013

COMPLETED:  125 / 1008  :  AJ Mitchell - Slow Dance Official Video ft Ava Max.mp3
COMPLETED:  126 / 1008  :  Ajab-si-Lyrics-From-the-Movie-(-Om-Shanti-Om)-Full-Song.mp3
COMPLETED:  127 / 1008  :  Ajeeb Dastan Hai Yeh Full Song HD With Lyrics - Dil Apna Aur Preet Parai.mp3
COMPLETED:  128 / 1008  :  Aji Rooth Kar Aab Kahan Jaiyaga - ARZOO - Copy.mp3
COMPLETED:  129 / 1008  :  Ajj Din Chadheya.mp3
COMPLETED:  130 / 1008  :  Ajnabi Thehro Zara - Taj Mahal An Eternal Love Story (2005) Full Song.mp3
COMPLETED:  131 / 1008  :  Akon -  Blame It On Me.mp3
COMPLETED:  132 / 1008  :  Akon - Belly Dancer.mp3
COMPLETED:  133 / 1008  :  Akon ft. Snoop Dogg - I Wanna Love You(Clean).mp3
COMPLETED:  134 / 1008  :  Akon-Don't Matter.mp3
COMPLETED:  135 / 1008  :  Akon-Lonely-Lyrics.mp3
COMPLETED:  136 / 1008  :  Ala Barfi.mp3
COMPLETED:  137 / 1008  :  Alan Walker ft. Au_Ra & Tomine Harket - Darkside.mp3
COMPLETED:  138 / 1008  :  Alane-Wes!!s.mp3
COMPLETED:  139 / 1008  :  Ale-golmaal3.mp3
COMPLETED:

...

COMPLETED:  627 / 1008  :  Melani.mp3
COMPLETED:  628 / 1008  :  Mera_Mann_Kehne_Laga_By_Falak_Nautanki_Saala_Song_Ayushmann_KhurranaKunaal_Roy_Kapur.mp3
COMPLETED:  629 / 1008  :  Meri Maa Lyrics  Main Kabhi Batlata Nahin   Taare Zameen Par  Shankar Mahadevan  Lyrics video.mp3
COMPLETED:  630 / 1008  :  Message To Bears - The Tourist.mp3
COMPLETED:  631 / 1008  :  Michael-Learns-To-Rock-Blue-Night.mp3
COMPLETED:  632 / 1008  :  Mike-Posner-I-Took-A-Pill-In-Ibiza-(Seeb-Remix)-(Explicit).mp3
COMPLETED:  633 / 1008  :  Mikky Ekko - Smile.mp3
COMPLETED:  634 / 1008  :  mirrors.mp3
COMPLETED:  635 / 1008  :  MIRZYA-Title-Song-MIRZYA-Rakeysh-Omprakash-Mehra-Gulzar-Shankar-Ehsaan-Loy-T-Series.mp3
COMPLETED:  636 / 1008  :  Mitwa - Mere Man Yeh Bata Day To With Lyrics By Team-4Teachcomflv.mp3
COMPLETED:  637 / 1008  :  Moh-Moh-Ke-Dhaage-(Female)-Song-with-Lyrics-Dum-Laga-Ke-Haisha-Ayushmann-Khurrana.mp3
COMPLETED:  638 / 1008  :  Mohobat ki jhooti kahani (MUGAL-E-AZAM 1960)_Misc.mp3
COMPLETED

C:\Users\pc\Anaconda3\lib\site-packages\librosa\core\pitch.py:146: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')
C:\Users\pc\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\pc\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\pc\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3367: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\Users\pc\Anaconda3\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\pc\Anaconda3\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


COMPLETED:  667 / 1008  :  New Mountain - 160512.mp3
COMPLETED:  668 / 1008  :  NewAgeTechno.mp3
COMPLETED:  669 / 1008  :  Nicki Minaj - Good Form ft Lil Wayne.mp3
COMPLETED:  670 / 1008  :  nigahein milane ko ji chahta hai .. asha bhosle..qawali..film dil hi to hai.mp3
COMPLETED:  671 / 1008  :  Nightingale.mp3
COMPLETED:  672 / 1008  :  Nimoli'-Dhruv-Ghanekar-Ila-Arun-Bobkat-Coke-Studio@MTV-Season-4.mp3
COMPLETED:  673 / 1008  :  NIT JYOT JAGUN.mp3
COMPLETED:  674 / 1008  :  Nobody's perfect.mp3
COMPLETED:  675 / 1008  :  Not In Love.mp3
COMPLETED:  676 / 1008  :  Nothing but thieves - If I get high.mp3
COMPLETED:  677 / 1008  :  Now no friends.mp3
COMPLETED:  678 / 1008  :  O baby.mp3
COMPLETED:  679 / 1008  :  O BECHARA DIL KYA KARE.mp3
COMPLETED:  680 / 1008  :  O BHALI BHALI SI EK SURAT.mp3
COMPLETED:  681 / 1008  :  O DILBAR DIL SE PYARE.mp3
COMPLETED:  682 / 1008  :  O DO NAINA AUR EK KAHANI.mp3
COMPLETED:  683 / 1008  :  O DO NAINON MEIN.mp3
COMPLETED:  684 / 1008  :  O DOLI 

COMPLETED:  797 / 1008  :  Show me your Jalwa - Aaja Nachle Lyrics.mp3
COMPLETED:  798 / 1008  :  SHREERAM CHANDRA KRIPALU - .MP3
COMPLETED:  799 / 1008  :  Shri Hanuman Chalisa.mp3
COMPLETED:  800 / 1008  :  Sin City 2 A Dame to Kill For Soundtrack #1 Main Theme (The Glitchmob - Cant Kill us).mp3
COMPLETED:  801 / 1008  :  Sleeping At Last - All Through The Night  [mp3clan.com].mp3
COMPLETED:  802 / 1008  :  Sleeping At Last - Saturn (Official Music Video).mp3
COMPLETED:  803 / 1008  :  Sleeping At Last - Snow lyrics HQ.mp3
COMPLETED:  804 / 1008  :  Sock My Dick.mp3
COMPLETED:  805 / 1008  :  Solar Fields - Sol.mp3
COMPLETED:  806 / 1008  :  Sooraj Dooba Hain Video Song _ Roy _ Arijit singh_Ranbir Kapoor _ Arjun Rampal _ Jacqueline.mp3
COMPLETED:  807 / 1008  :  Sound-Of-Legend-Blue-(Da-Ba-dee)-(Official-Video).mp3
COMPLETED:  808 / 1008  :  Sri Ramachandra Kripalu - Sooryagayathri - 'Vande Guru Paramparaam'.mp3
COMPLETED:  809 / 1008  :  Starboy-weekend.mp3
COMPLETED:  810 / 1008  :

COMPLETED:  922 / 1008  :  Troye Sivan - Happy Little Pill.mp3
COMPLETED:  923 / 1008  :  Troye Sivan - Lost Boy.mp3
COMPLETED:  924 / 1008  :  Troye Sivan - Strawberries  Cigarettes.mp3
COMPLETED:  925 / 1008  :  Troye Sivan - SUBURBIA.mp3
COMPLETED:  926 / 1008  :  Troye Sivan - Swimming Pools.mp3
COMPLETED:  927 / 1008  :  Troye Sivan - Talk Me Down.mp3
COMPLETED:  928 / 1008  :  Troye Sivan - TOO GOOD.mp3
COMPLETED:  929 / 1008  :  Troye Sivan - WILD.mp3
COMPLETED:  930 / 1008  :  Troye Sivan - Youth.mp3
COMPLETED:  931 / 1008  :  Troye-Sivan - My My My!.mp3
COMPLETED:  932 / 1008  :  Tu na jaane aas pass hai khuda.mp3
COMPLETED:  933 / 1008  :  Tujhe bhula diya.mp3
COMPLETED:  934 / 1008  :  Tujhe yaad kar liya hai aayat ki tarah.mp3
COMPLETED:  935 / 1008  :  Tum Itna Jo Muskura Rahe Ho  Jagjit Singh  Arth 1983 Songs  Ghazal Song.mp3
COMPLETED:  936 / 1008  :  TUM JO AAYE.mp3
COMPLETED:  937 / 1008  :  TUM KO DEKHA TO YEH KAYAL.mp3
COMPLETED:  938 / 1008  :  Tum Ko Dekha Toh Ye K